## Generate a JSON events sidecar template and spreadsheet from a BIDS dataset

This generates a JSON events sidecar template and a corresponding 4-column spreadsheet
for the BCIT Basic Guard Duty data.

For tutorial examples of this process see the Jupyter notebooks:
- [`bids_generate_sidecar.ipynb`](https://github.com/hed-standard/hed-examples/blob/main/hedcode/jupyter_notebooks/bids_processing/bids_generate_sidecar.ipynb)
- [`bids_sidecar_to_spreadsheet.ipynb`](https://github.com/hed-standard/hed-examples/blob/main/hedcode/jupyter_notebooks/bids_processing/bids_sidecar_to_spreadsheet.ipynb)


In [1]:
import os
import datetime
import json
from hed.tools import hed_to_df, BidsTsvSummary, HedLogger
from hed.util import get_file_list

# Variables to set for the specific dataset
bids_root_path = 's:/bcit/BasicGuardDutyWorkingPhaseTwo'
json_path = os.path.realpath(os.path.join(bids_root_path, 'task-GuardDuty_events.json'))
spreadsheet_path = os.path.realpath(os.path.join(bids_root_path, 'code/task-GuardDuty_hed_spreadsheet.tsv'))
exclude_dirs = ['sourcedata', 'stimuli', 'code']
entities = ('sub', 'ses', 'run')
skip_columns = ['onset', 'duration', 'sample']
value_columns = ['gid']
log_name = 'bcit_basic_guard_duty_04_generate_sidecar_log'

# Set up the logger
log_file_name = f"code/curation_logs/{log_name}.txt"
logger = HedLogger(name=log_name)

# Construct the list of .event files
bids_files = get_file_list(bids_root_path, extensions=[".tsv"], name_suffix="_events", exclude_dirs=exclude_dirs)

# Construct the event file value summary and generate a sidecar template representing dataset
logger.add("overall", f"Summarizing {len(bids_files)}")
value_summary = BidsTsvSummary(value_cols=value_columns, skip_cols=skip_columns, name="BCIT Basic Guard Duty")
value_summary.update(bids_files)
sidecar_template = value_summary.extract_sidecar_template()
with open(json_path, 'w') as fp:
   json.dump(sidecar_template, fp, indent=4)
logger.add("overall", f"Saving a JSON sidecar template to {os.path.basename(json_path)}")

hed_spreadsheet = hed_to_df(sidecar_template)
logger.add("overall", f"Creating a spreadsheet template and saving to {os.path.basename(spreadsheet_path)}")
hed_spreadsheet.to_csv(spreadsheet_path, sep='\t', index=False,)

# Output and save the log
log_string = "\n\nLog output:\n" + logger.get_log_string()
error_string = "\n\nERROR Summary:\n" + logger.get_log_string(level="ERROR")
print(log_string)
print(error_string)

save_path = os.path.join(bids_root_path, log_file_name)
with open(save_path, "w") as fp:
    fp.write(f"{log_file_name} {datetime.datetime.now()}\n")
    fp.write(log_string)
    fp.write(error_string)



Log output:
bcit_basic_guard_duty_04_generate_sidecar_log: Level None
overall:
	[ Summarizing 22]
	[ Saving a JSON sidecar template to task-GuardDuty_events.json]
	[ Creating a spreadsheet template and saving to task-GuardDuty_hed_spreadsheet.tsv]


ERROR Summary:
bcit_basic_guard_duty_04_generate_sidecar_log: Level ERROR
overall:
